In [3]:
import datetime
import csv
import elasticsearch
import elasticsearch.helpers
from collections import OrderedDict


DATE = "20190816"


ES_ADDR = ["10.0.6.7:9200"]

q = {
    "query": {
        "bool": {
            "must": [
                {"term": {"log_type": "credit"}},
                {"term": {"result": "unkonwn"}},
                {"term": {"extra.ref_res": ""}}
            ],
            "must_not": {
                "term": {"af_result": "reject"}
            }
        }
    }
}

headers = ["create_time", "trx", "log_type", "result", "err", "has_af", "af_result", "af_err", "ref_res", "name", "phone", "uin", 
           "bussiness_code", "merchant_code", "production_code"]

name = "online_report_credit_%s.csv" % DATE
with open(name, "w", newline="", encoding='utf-8-sig') as f:
    w = csv.DictWriter(f, fieldnames=headers)
    w.writeheader()

    es = elasticsearch.Elasticsearch(hosts=ES_ADDR)
    results = elasticsearch.helpers.scan(es, index="obs-%s" % DATE, query=q)
    for item in results:
        tmp_dict = {}
        for key in headers:
            try:
                value = item["_source"].get(key, "")
                tmp_dict[key] = value
            except Exception as e:
                if key in ["af_result", "af_err"]:
                    tmp_dict[key] = ""
                elif key == "ref_res":
                    tmp_dict["ref_res"] = item["_source"].get("extra", {}).get("ref_res", "")
                else:
                    print("key:", key, item["_source"])
                    continue
        tmp_dict["create_time"] = datetime.datetime.fromtimestamp(tmp_dict["create_time"]/1000).strftime("%Y-%m-%d %H:%M:%S")
        w.writerow(tmp_dict)
print("success...")

success...
